# Workshop Setup - User Provisioning and Catalog Management

This notebook automates the provisioning of Unity Catalog resources for multiple users.

## Prerequisites
- Delta Share must be created and shared in Databricks BEFORE running this notebook
- You need the provider name and share name from your Delta Share setup

## Workflow
1. Load configuration from config.yaml
2. Parse user list and generate user aliases
3. Mount Delta Share to local catalog (format: `provider.share`)
4. Create user-specific catalogs with naming convention
5. Assign CAN MANAGE permissions to catalog owners
6. Create volumes with consistent naming
7. Load data from Delta Share volume to user volumes
8. Generate provisioning report (CSV + JSON)

## Configuration Pattern
- Config values loaded from config.yaml
- Widget-based parameter override capability
- Type-safe parameter handling
- Clear execution flow with status reporting

## 0. Install Dependencies

Install required packages from requirements.txt.

In [0]:
%pip install -q -r ../requirements.txt
dbutils.library.restartPython()

## 1. Load Configuration from config.yaml

Load configuration file and use values as widget defaults.

In [0]:
import yaml
import os

# Load configuration from config.yaml
# Support multiple execution contexts (local, workspace, etc.)
config_paths = [
    "../config.yaml",  # Relative path from notebooks/ directory
    # "/Workspace/Repos/dbx-sdp-workshop/config.yaml",  # Workspace path
    # "config.yaml"  # Current directory fallback
]

config = None
config_loaded_from = None

for config_path in config_paths:
    try:
        with open(config_path, "r") as f:
            config = yaml.safe_load(f)
            config_loaded_from = config_path
            break
    except FileNotFoundError:
        continue
    except Exception as e:
        print(f"⚠️  Error reading config from {config_path}: {str(e)}")
        continue

if config is None:
    print("⚠️  Warning: Could not load config.yaml, using default values")
    config = {}
else:
    print(f"✅ Configuration loaded from: {config_loaded_from}")
    print(f"   Configuration keys: {list(config.keys())}")

## 2. Create Configuration Widgets

Create widgets with defaults from config.yaml.

In [0]:
# Widget Configuration
# Config.yaml provides default values for all widgets

# User Configuration
dbutils.widgets.text("user_list", config.get("user_list", "marcin.jimenez@databricks.com"), "User List (comma-separated emails)")

# Delta Share Configuration (Databricks-to-Databricks)
# NOTE: Delta Share must be created and shared BEFORE running this notebook
# Provider format: Use EXACT name from "SHOW PROVIDERS;" (e.g., "azure:eastus2:databricks:field-eng-east")
dbutils.widgets.text("delta_share_provider", config.get("delta_share_provider", "azure:eastus2:databricks:field-eng-east"), "Delta Share Provider (from SHOW PROVIDERS)")
dbutils.widgets.text("delta_share_name", config.get("delta_share_name", "scp-demo"), "Delta Share Name (from SHOW SHARES)")
dbutils.widgets.text("delta_share_catalog", config.get("delta_share_catalog", "shared_catalog"), "Delta Share Catalog Name (local)")
dbutils.widgets.text("delta_share_schema", config.get("delta_share_schema", "shared_schema"), "Delta Share Schema Name")
dbutils.widgets.text("delta_share_volume", config.get("delta_share_volume", "shared_volume"), "Delta Share Volume Name")

# File Pattern Configuration
dbutils.widgets.text("file_glob_pattern", config.get("file_glob_pattern", "*"), "File Glob Pattern to Copy")

# Catalog Configuration
dbutils.widgets.text("base_catalog_name", config.get("base_catalog_name", "workshop_catalog"), "Base Catalog Name")

# Volume Configuration
dbutils.widgets.text("volume_name", config.get("volume_name", "user_data_volume"), "Volume Name (consistent across all catalogs)")
dbutils.widgets.text("schema_name", config.get("schema_name", "default"), "Schema Name for Volumes")

print("✅ Configuration widgets created successfully")

## 3. Load and Validate Configuration

In [0]:
# Retrieve widget values (widgets override config.yaml if changed by user)
user_list_raw = dbutils.widgets.get("user_list")
delta_share_provider = dbutils.widgets.get("delta_share_provider")
delta_share_name = dbutils.widgets.get("delta_share_name")
delta_share_catalog = dbutils.widgets.get("delta_share_catalog")
delta_share_schema = dbutils.widgets.get("delta_share_schema")
delta_share_volume = dbutils.widgets.get("delta_share_volume")
file_glob_pattern = dbutils.widgets.get("file_glob_pattern")
base_catalog_name = dbutils.widgets.get("base_catalog_name")
volume_name = dbutils.widgets.get("volume_name")
schema_name = dbutils.widgets.get("schema_name")

# Display configuration
print("🔧 Workshop Provisioning Configuration:")
print("=" * 80)
print(f"Config Source:       {config_loaded_from or 'defaults'}")
print(f"User List:           {user_list_raw}")
print(f"Delta Share:         {delta_share_provider}.{delta_share_name}")
print(f"Delta Share Catalog: {delta_share_catalog} (mounting to local workspace)")
print(f"Delta Share Source:  {delta_share_catalog}.{delta_share_schema}.{delta_share_volume}")
print(f"File Glob Pattern:   {file_glob_pattern}")
print(f"Base Catalog:        {base_catalog_name}")
print(f"Volume Name:         {volume_name}")
print(f"Schema Name:         {schema_name}")
print("=" * 80)

## 4. Parse Users and Generate Aliases

Parse the comma-separated user list and generate user aliases using the pattern:
- First 3 letters of first name
- First 4 letters of last name
- Concatenated with underscore
- Example: "John Smith" → "joh_smit"

In [0]:
import re

def generate_user_alias(email):
    """
    Generate user alias from email address.
    Pattern: first 3 letters of first name + _ + first 4 letters of last name
    
    Args:
        email: Email address (e.g., "marcin.jimenez@databricks.com")
    
    Returns:
        User alias (e.g., "marjime")
    """
    # Extract username part from email (before @)
    if '@' not in email:
        raise ValueError(f"Invalid email format: {email}. Expected 'first.last@domain.com'")
    
    username = email.split('@')[0]
    
    # Split by dot or underscore to get first and last name
    name_parts = re.split(r'[._]', username)
    
    if len(name_parts) < 2:
        raise ValueError(f"Invalid email format: {email}. Expected 'first.last@domain.com' or 'first_last@domain.com'")
    
    first_name = name_parts[0].lower()
    last_name = name_parts[-1].lower()  # Use last part in case of middle names
    
    # Generate alias: first 3 chars of first name + first 4 chars of last name
    first_part = first_name[:3]
    last_part = last_name[:4]
    
    alias = f"{first_part}{last_part}"
    
    # Remove any non-alphanumeric characters except underscore
    alias = re.sub(r'[^a-z0-9_]', '', alias)
    
    return alias

def extract_full_name(email):
    """
    Extract full name from email address.
    
    Args:
        email: Email address (e.g., "marcin.jimenez@databricks.com")
    
    Returns:
        Full name with proper capitalization (e.g., "Marcin Jimenez")
    """
    username = email.split('@')[0]
    name_parts = re.split(r'[._]', username)
    
    # Capitalize each part
    full_name = ' '.join(part.capitalize() for part in name_parts)
    
    return full_name

# Parse user list (expecting comma-separated email addresses)
users = [user.strip() for user in user_list_raw.split(',') if user.strip()]

# Validate and generate user data structure
user_data = []
errors = []

for email in users:
    try:
        alias = generate_user_alias(email)
        full_name = extract_full_name(email)
        
        user_data.append({
            "full_name": full_name,
            "alias": alias,
            "catalog_name": f"{base_catalog_name}_{alias}",
            "email": email
        })
    except ValueError as e:
        errors.append(str(e))

# If there are any errors, raise an exception
if errors:
    error_message = "❌ User parsing failed:\n" + "\n".join(f"   - {err}" for err in errors)
    raise ValueError(error_message)

# Display parsed users
print(f"\n📋 Parsed {len(user_data)} users:")
print("=" * 80)
for idx, user in enumerate(user_data, 1):
    print(f"{idx}. {user['email']:40s} → {user['full_name']:20s} → Alias: {user['alias']:12s} → Catalog: {user['catalog_name']}")
print("=" * 80)

## 5. Mount Delta Share to Catalog

Mount the Delta Share to a local catalog for accessing shared data.

In [0]:
from databricks.sdk import WorkspaceClient

def mount_delta_share_catalog(catalog_name, provider, share_name):
    try:
        # Check if catalog already exists
        try:
            spark.sql(f"DESCRIBE CATALOG `{catalog_name}`")
            print(f"✅ Delta Share catalog '{catalog_name}' already mounted")
            return {"status": "already_exists", "mounted": True, "catalog": catalog_name}
        except Exception:
            pass
        
        w = WorkspaceClient()

        # Create catalog from Delta Share
        catalog = w.catalogs.create(
            name=catalog_name,
            provider_name=provider,
            share_name=share_name,
            comment="Catalog created for SCP Workshop"  # optional
        )
        
        print(f"✅ Mounted Delta Share '{share_name}' to catalog '{catalog_name}'")
        return {"status": "success", "mounted": True, "catalog": catalog_name}
        
    except Exception as e:
        print(f"❌ Error mounting Delta Share: {str(e)}")
        print(f"   Verify share exists: {provider}.{share_name}")
        print(f"   Run 'SHOW PROVIDERS;' and 'SHOW SHARES;' to verify names")
        raise e

# Mount Delta Share
print("\n📦 Mounting Delta Share to Catalog:")
print("=" * 80)
mount_status = mount_delta_share_catalog(delta_share_catalog, delta_share_provider, delta_share_name)
print("=" * 80)

## 6. Create User Catalogs

Create Unity Catalog catalogs for each user with the naming pattern: `{base_catalog_name}_{user_alias}`

In [0]:
def create_catalog(catalog_name, comment):
    """Create a Unity Catalog catalog."""
    try:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS `{catalog_name}` COMMENT '{comment}'")
        return {"catalog": catalog_name, "created": True, "status": "success"}
    except Exception as e:
        print(f"❌ Error creating catalog {catalog_name}: {str(e)}")
        return {"catalog": catalog_name, "created": False, "status": "error", "error": str(e)}

# Create catalogs for all users
print("\n📚 Creating User Catalogs:")
print("=" * 80)

catalog_results = []
for user in user_data:
    comment = f"Catalog for user {user['full_name']} created: {user['catalog_name']}"
    result = create_catalog(user['catalog_name'], comment)
    catalog_results.append(result)
    user['catalog_created'] = result['created']
    print(comment)

print("=" * 80)
success_count = sum(1 for r in catalog_results if r.get('created', False))
print(f"📊 Created {success_count}/{len(catalog_results)} catalogs")

## 7. Assign CAN MANAGE Permissions

Grant CAN MANAGE permissions to each user for their respective catalog.

In [0]:
def grant_catalog_permissions(catalog_name, user_email):
    """Grant CAN MANAGE permissions on a catalog to a user."""
    try:
        for priv in ["USE CATALOG", "USE SCHEMA", "CREATE SCHEMA"]:
            spark.sql(f"GRANT {priv} ON CATALOG `{catalog_name}` TO `{user_email}`")
        
        spark.sql(f"GRANT ALL PRIVILEGES ON CATALOG `{catalog_name}` TO `{user_email}`")
        print(f"Successful granting permissions on {catalog_name} to {user_email}")
        return {"catalog": catalog_name, "user": user_email, "granted": True, "status": "success"}
        
    except Exception as e:
        print(f"❌ Error granting permissions on {catalog_name} to {user_email}: {str(e)}")
        return {"catalog": catalog_name, "user": user_email, "granted": False, "status": "error", "error": str(e)}

# Grant permissions for all users
print("\n🔐 Assigning CAN MANAGE Permissions:")
print("=" * 80)

permission_results = []
for user in user_data:
    result = grant_catalog_permissions(user['catalog_name'], user['email'])
    permission_results.append(result)
    user['permissions_granted'] = result['granted']

print("=" * 80)
success_count = sum(1 for r in permission_results if r.get('granted', False))
print(f"📊 Granted permissions to {success_count}/{len(permission_results)} catalogs")

## 8. Create Volumes with Consistent Naming

Create a volume in each catalog with a consistent name across all catalogs.

In [0]:
def create_schema_and_volume(catalog_name, schema_name, volume_name):
    """Create schema and volume in a catalog."""
    try:
        volume_path = f"{catalog_name}.{schema_name}.{volume_name}"
        
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{catalog_name}`.`{schema_name}` COMMENT 'Schema for user data and volumes'")
        spark.sql(f"CREATE VOLUME IF NOT EXISTS `{catalog_name}`.`{schema_name}`.`{volume_name}` COMMENT 'User data volume'")
        print(f"Successful creating volume in {catalog_name}.{schema_name}.{volume_name}")
        return {"catalog": catalog_name, "volume_path": volume_path, "created": True, "status": "success"}
        
    except Exception as e:
        print(f"❌ Error creating volume in {catalog_name}: {str(e)}")
        return {"catalog": catalog_name, "volume_path": volume_path, "created": False, "status": "error", "error": str(e)}

# Create volumes for all users
print("\n💾 Creating User Volumes:")
print("=" * 80)

volume_results = []
for user in user_data:
    result = create_schema_and_volume(user['catalog_name'], schema_name, volume_name)
    volume_results.append(result)
    user['volume_name'] = result['volume_path']
    user['volume_created'] = result['created']

print("=" * 80)
success_count = sum(1 for r in volume_results if r.get('created', False))
print(f"📊 Created {success_count}/{len(volume_results)} volumes")

## 9. Load Data to Volumes from Delta Share

Copy data from Delta Share volume to each user's volume using the glob pattern.

In [0]:
import fnmatch

def list_files_recursive(path):
    """Recursively list all files in a path."""
    all_files = []
    
    def traverse(current_path):
        try:
            items = dbutils.fs.ls(current_path)
            for item in items:
                if item.isDir():
                    # Recursively traverse directories
                    traverse(item.path)
                else:
                    # Add files to the list
                    all_files.append(item)
        except Exception as e:
            print(f"      ⚠️  Error accessing {current_path}: {str(e)}")
    
    traverse(path)
    return all_files

def normalize_path(path):
    """Remove dbfs: prefix and normalize path."""
    return path.replace('dbfs:', '').replace('//', '/')

def load_data_to_volume(volume_path, delta_share_catalog, delta_share_schema, delta_share_volume, file_pattern):
    """Copy data from Delta Share volume to user volume using glob pattern."""
    try:
        # Root destination - no /data subdirectory
        volume_fs_path = f"/Volumes/{volume_path.replace('.', '/')}"
        
        # Access Delta Share volume
        delta_share_volume_path = f"/Volumes/{delta_share_catalog}/{delta_share_schema}/{delta_share_volume}"
        
        print(f"   📁 Source: {delta_share_volume_path}")
        print(f"   📁 Destination: {volume_fs_path}")
        print(f"   🔍 Pattern: {file_pattern}")
        
        # List and filter files recursively
        try:
            print(f"   🔄 Scanning directories recursively...")
            all_files = list_files_recursive(delta_share_volume_path)
            print(f"   📋 Found {len(all_files)} files in source (recursive)")
            
            # Debug: Show all files found
            for f in all_files:
                print(f"      - {f.name} (matches pattern: {fnmatch.fnmatch(f.name, file_pattern)})")
            
            matching_files = [f for f in all_files if fnmatch.fnmatch(f.name, file_pattern)]
            
            if not matching_files:
                print(f"   ⚠️  No files match pattern '{file_pattern}'")
                return {"volume": volume_path, "data_location": volume_fs_path, "loaded": False, "status": "no_files_found", "files_copied": 0}
            
            print(f"   ✓ {len(matching_files)} files match pattern")
            
            # Copy files preserving directory structure
            files_copied = 0
            errors = []
            for file_obj in matching_files:
                # Normalize paths to remove dbfs: prefix
                normalized_file_path = normalize_path(file_obj.path)
                normalized_source_path = normalize_path(delta_share_volume_path)
                
                # Get relative path from source
                relative_path = normalized_file_path.replace(normalized_source_path, '').lstrip('/')
                dest_path = f"{volume_fs_path}/{relative_path}"
                
                # print(f"      🔍 Debug: {normalized_file_path} -> {dest_path}")
                
                # Create destination directory if needed
                dest_dir = '/'.join(dest_path.split('/')[:-1])
                try:
                    dbutils.fs.mkdirs(dest_dir)
                except:
                    pass  # Directory might already exist
                
                try:
                    dbutils.fs.cp(file_obj.path, dest_path)
                    files_copied += 1
                    print(f"      ✓ Copied: {relative_path}")
                except Exception as e:
                    error_msg = f"Failed to copy {relative_path}: {str(e)}"
                    errors.append(error_msg)
                    print(f"      ✗ {error_msg}")
            
            if errors:
                print(f"   ⚠️  Encountered {len(errors)} errors during copy")
            
            print(f"   ✅ Successfully loaded data to {volume_path} ({files_copied} files)")
            
            return {
                "volume": volume_path,
                "data_location": volume_fs_path,
                "loaded": True if files_copied > 0 else False,
                "status": "success" if files_copied > 0 else "copy_errors",
                "files_copied": files_copied,
                "source": "delta_share",
                "errors": errors if errors else None
            }
            
        except Exception as e:
            print(f"   ❌ Error listing files: {str(e)}")
            raise e
            
    except Exception as e:
        print(f"❌ Error loading data to {volume_path}: {str(e)}")
        raise e

# Load data to all volumes
print("\n📥 Loading Data to Volumes:")
print("=" * 80)

data_load_results = []
for user in user_data:
    print(f"\n👤 Processing user: {user['full_name']} ({user['alias']})")
    
    if not user.get('volume_created', False):
        print(f"   ⚠️  Skipping - volume was not created successfully")
        user['data_location'] = None
        user['data_loaded'] = False
        user['files_copied'] = 0
        continue
    
    result = load_data_to_volume(
        user['volume_name'],
        delta_share_catalog,
        delta_share_schema,
        delta_share_volume,
        file_glob_pattern
    )
    data_load_results.append(result)
    user['data_location'] = result['data_location']
    user['data_loaded'] = result['loaded']
    user['files_copied'] = result.get('files_copied', 0)

print("\n" + "=" * 80)
success_count = sum(1 for r in data_load_results if r.get('loaded', False))
total_files = sum(r.get('files_copied', 0) for r in data_load_results)

print(f"📊 Loaded data to {success_count}/{len(data_load_results)} volumes")
print(f"   Total files copied: {total_files}")

# Show any errors
errors_found = [r for r in data_load_results if r.get('errors')]
if errors_found:
    print(f"\n⚠️  {len(errors_found)} volumes had copy errors - see details above")

## 10. Generate Provisioning Report Table

Create a comprehensive report table showing all provisioning details.

In [0]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, BooleanType

# Define schema for the report
report_schema = StructType([
    StructField("user_id", StringType(), False),
    StructField("full_name", StringType(), False),
    StructField("user_alias", StringType(), False),
    StructField("catalog_created", StringType(), False),
    StructField("permissions_assigned", BooleanType(), False),
    StructField("volume_name", StringType(), True),
    StructField("volume_data_location", StringType(), True),
    StructField("provisioning_status", StringType(), False)
])

# Build report rows
report_rows = []
for idx, user in enumerate(user_data, 1):
    # Determine overall provisioning status
    if user.get('catalog_created', False) and user.get('permissions_granted', False) and user.get('volume_created', False) and user.get('data_loaded', False):
        status = "✅ Complete"
    else:
        status = "⚠️ Partial"
    
    report_rows.append(
        Row(
            user_id=user['email'],
            full_name=user['full_name'],
            user_alias=user['alias'],
            catalog_created=user['catalog_name'],
            permissions_assigned=user.get('permissions_granted', False),
            volume_name=user.get('volume_name', None),
            volume_data_location=user.get('data_location', None),
            provisioning_status=status
        )
    )

# Create DataFrame
report_df = spark.createDataFrame(report_rows, schema=report_schema)

# Display report
print("\n📊 Provisioning Report:")
print("=" * 80)
display(report_df)

## 11. Summary Statistics

In [0]:
# Calculate summary statistics
total_users = len(user_data)
catalogs_created = sum(1 for u in user_data if u.get('catalog_created', False))
permissions_granted = sum(1 for u in user_data if u.get('permissions_granted', False))
volumes_created = sum(1 for u in user_data if u.get('volume_created', False))
data_loaded = sum(1 for u in user_data if u.get('data_loaded', False))

# Display summary
print("\n" + "=" * 80)
print("📈 WORKSHOP PROVISIONING SUMMARY")
print("=" * 80)
print(f"Total Users Processed:        {total_users}")
print(f"Catalogs Created:             {catalogs_created}/{total_users} ({catalogs_created/total_users*100:.1f}%)")
print(f"Permissions Granted:          {permissions_granted}/{total_users} ({permissions_granted/total_users*100:.1f}%)")
print(f"Volumes Created:              {volumes_created}/{total_users} ({volumes_created/total_users*100:.1f}%)")
print(f"Data Loaded:                  {data_loaded}/{total_users} ({data_loaded/total_users*100:.1f}%)")
print("=" * 80)

# Success check
if catalogs_created == total_users and permissions_granted == total_users and volumes_created == total_users and data_loaded == total_users:
    print("\n✅ SUCCESS: All users provisioned successfully!")
else:
    print("\n⚠️  WARNING: Some provisioning steps failed. Review the report above for details.")
    
print("=" * 80)

## 12. Export Report to CSV

Save the provisioning report to a CSV file for tracking and cleanup.

In [0]:
# Save report to CSV and JSON for cleanup tracking
report_csv_file = "../provisioning_report.csv"
report_json_file = "../provisioning_report.json"

try:
    import json
    from datetime import datetime
    
    # Save JSON report for cleanup operations
    report_data = {
        "base_catalog_name": base_catalog_name,
        "users": user_data,
        "timestamp": datetime.now().isoformat()
    }
    
    with open(report_json_file, 'w') as f:
        json.dump(report_data, f, indent=2)
    
    print(f"💾 JSON report saved to: {report_json_file}")
    
    # Save CSV report for easy viewing
    report_df.toPandas().to_csv(report_csv_file, index=False)
    print(f"💾 CSV report saved to: {report_csv_file}")
    
except Exception as e:
    print(f"⚠️  Could not save reports: {str(e)}")

---

## Cleanup Section - Delete Provisioned Resources

⚠️ **WARNING**: The following cells will DELETE all catalogs created by this workshop setup.

**Instructions:**
1. Run the first cell to load the provisioning report
2. Run the second cell and type "CONFIRM" in the widget to delete all resources

In [0]:
import json
import os

# Try to load the provisioning report
report_file_paths = [
    "../provisioning_report.json",
    "/Workspace/Repos/dbx-sdp-workshop/provisioning_report.json",
    "provisioning_report.json"
]

cleanup_data = None
report_file_loaded = None

for path in report_file_paths:
    try:
        with open(path, 'r') as f:
            cleanup_data = json.load(f)
            report_file_loaded = path
            break
    except FileNotFoundError:
        continue
    except Exception as e:
        print(f"⚠️  Error reading {path}: {str(e)}")
        continue

if cleanup_data is None:
    print("❌ No provisioning report found!")
    print("   Run the provisioning cells above first to create resources.")
    print("   The report file (provisioning_report.json) is required for cleanup.")
else:
    print(f"✅ Provisioning report loaded from: {report_file_loaded}")
    print(f"   Base catalog name: {cleanup_data.get('base_catalog_name')}")
    print(f"   Users found: {len(cleanup_data.get('users', []))}")
    print(f"   Timestamp: {cleanup_data.get('timestamp', 'N/A')}")
    
    # Display catalogs to be deleted
    print(f"\n📋 Catalogs that will be deleted:")
    print("=" * 80)
    for idx, user in enumerate(cleanup_data.get('users', []), 1):
        print(f"{idx}. {user['catalog_name']}")
    print("=" * 80)

In [0]:
# Create confirmation widget if it doesn't exist

# deletion_results = []

# for user in cleanup_data.get('users', []):
#     catalog_name = user.get('catalog_name')
    
#     try:
#         # Drop the catalog (CASCADE will delete all contents)
#         spark.sql(f"DROP CATALOG IF EXISTS `{catalog_name}` CASCADE")
#         print(f"   ✅ Deleted catalog: {catalog_name}")
#         deletion_results.append({
#             "catalog": catalog_name,
#             "deleted": True,
#             "status": "success"
#         })
        
#     except Exception as e:
#         print(f"   ❌ Error deleting catalog {catalog_name}: {str(e)}")
#         deletion_results.append({
#             "catalog": catalog_name,
#             "deleted": False,
#             "status": "error",
#             "error": str(e)
#         })

# print("=" * 80)

# # Display summary
# success_count = sum(1 for r in deletion_results if r.get('deleted', False))
# total_count = len(deletion_results)

# print(f"\n📊 Deletion Summary:")
# print(f"   Catalogs deleted: {success_count}/{total_count}")